In [1]:
from scipy.optimize import minimize, NonlinearConstraint, basinhopping
import time
import numpy as np

Our additional requirements will be:
- optimise expected_value (prob * risk)
- vector/list of parameters (5 skill entries for each worker -> length = 5*N)
- bound contraints [0,1] for each element
- sum of worker contributions (5 elements) <= how many units they have free to contribute 
- departmental workload is met (This is the hard one because it is dynamic: the constraint function will have to be re-evaluated with each assignment tested by the otpimisation routin. *Can Gekko handle this?* - it appears so from the above, but if not we may need to approximate the workload constraint.)
- add team size constraint

In [2]:
import sys, os
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [3]:
from superscript_model import model
from superscript_model.utilities import Random
from superscript_model.project import Project
from superscript_model.organisation import Team
from superscript_model.config import MIN_TEAM_SIZE, MAX_TEAM_SIZE

In [4]:
STEPS = 25

abm = model.SuperScriptModel(worker_count=60, 
                             new_projects_per_timestep=2, 
                             worker_strategy = "Stake",
                             organisation_strategy = 'Basic')

In [5]:
abm.run_model(STEPS)

In [6]:
tracked = abm.datacollector.get_model_vars_dataframe()
tracked.head(STEPS)

,ActiveProjects,RecentSuccessRate,SuccessfulProjects,FailedProjects,NullProjects,WorkersOnProjects,WorkersWithoutProjects,WorkersOnTraining,AverageTeamSize,AverageSuccessProbability,AverageWorkerOvr,AverageTeamOvr,WorkerTurnover,ProjectLoad,TrainingLoad,DeptLoad,Slack,ProjectsPerWorker
0,2,0.000000,0.0,0.0,0,7,53,0,6.000000,0.437868,50.283794,65.085457,0.0,0.070000,0.000000,0.1,0.830000,0.200000
1,4,0.000000,0.0,0.0,0,16,44,0,6.500000,0.237818,49.933252,57.905457,0.0,0.138333,0.000000,0.1,0.761667,0.433333
2,5,0.000000,0.0,0.0,1,19,41,0,6.600000,0.205980,49.612468,57.882660,0.0,0.171667,0.000000,0.1,0.728333,0.550000
3,7,0.000000,0.0,0.0,0,21,39,0,6.285714,0.174176,49.307657,56.744001,0.0,0.218333,0.000000,0.1,0.681667,0.733333
4,7,0.058333,1.0,1.0,0,26,34,0,6.285714,0.144045,49.555776,52.259250,0.0,0.208333,0.000000,0.1,0.691667,0.733333
5,6,0.044444,0.0,2.0,0,24,36,0,5.666667,0.159316,49.268023,46.855522,0.0,0.148333,0.000000,0.1,0.751667,0.533333
6,5,0.044444,0.0,1.0,1,22,38,0,5.400000,0.175454,48.924763,44.668332,0.0,0.115000,0.000000,0.1,0.785000,0.416667
7,7,0.083333,1.0,1.0,0,22,38,0,6.000000,0.171622,48.959948,47.782177,0.0,0.198333,0.000000,0.1,0.701667,0.666667
8,8,0.077778,0.0,1.0,0,25,35,0,6.375000,0.189496,48.692429,49.906369,0.0,0.246667,0.000000,0.1,0.653333,0.816667
9,7,0.072222,0.0,2.0,1,21,37,2,6.714286,0.169536,52.346588,54.487007,25.0,0.243333,0.033333,0.1,0.623333,0.783333


#### Now, with the current state of the system, we will try creating a new project and finding its optimal team allocation:

In [12]:
project = Project(abm.inventory, 
                  abm.inventory.index_total + 1,
                  project_length=5,
                  start_time_offset=abm.inventory.get_start_time_offset(),
                  start_time=abm.schedule.steps)

In [13]:
print(project.requirements.to_string())

{
    "risk": 5,
    "creativity": 2,
    "flexible_budget": false,
    "budget": 7,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 4,
    "hard_skills": {
        "A": {
            "level": 4,
            "units": 1
        },
        "B": {
            "level": 1,
            "units": 1
        },
        "C": {
            "level": null,
            "units": 0
        },
        "D": {
            "level": null,
            "units": 0
        },
        "E": {
            "level": 1,
            "units": 2
        }
    }
}


In [14]:
bid_pool = abm.inventory.team_allocator.strategy.invite_bids(project)
print(len(bid_pool))

50


In [15]:
x = np.zeros(len(bid_pool) * 5)
x0 = np.zeros(len(x))
bounds = list([(0,1) for i in x])

In [16]:
print(MAX_TEAM_SIZE) # = 7
print(MIN_TEAM_SIZE) # = 3

7
3


#### We now add the constraint on the number uf units that each worker can contribute to the project (worker_unit_budget):

In [17]:
# project_1 = project
# project = project_2

In [18]:
worker_unit_budgets = {
    worker.worker_id: worker.contributions.get_remaining_units(
        project.start_time, project.length
    )
    for worker in bid_pool
}
print(min(worker_unit_budgets.values()))
print(max(worker_unit_budgets.values()))

worker_ids = list(worker_unit_budgets.keys())
worker_unit_budgets = worker_unit_budgets

1
10


In [19]:
worker_unit_budgets

{2: 10,
 3: 4,
 7: 8,
 9: 3,
 11: 5,
 14: 10,
 25: 8,
 27: 7,
 29: 3,
 34: 5,
 37: 1,
 45: 3,
 55: 3,
 58: 6,
 64: 9,
 68: 4,
 80: 5,
 84: 10,
 85: 8,
 87: 10,
 88: 10,
 89: 10,
 90: 10,
 91: 10,
 92: 10,
 93: 8,
 94: 10,
 95: 10,
 96: 10,
 97: 3,
 98: 10,
 99: 10,
 100: 10,
 101: 10,
 102: 10,
 103: 3,
 104: 10,
 105: 10,
 106: 10,
 107: 10,
 108: 10,
 109: 10,
 110: 10,
 111: 10,
 112: 7,
 113: 10,
 114: 9,
 115: 10,
 116: 10,
 117: 10}

In [20]:
constraints = []
constraints_slsqp = []

for wi, worker in enumerate(bid_pool):
    start = wi*5
    constraints.append(
        NonlinearConstraint(lambda x: sum(np.round(x[start:start+5])) - worker_unit_budgets[worker.worker_id], -np.inf, 0)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: worker_unit_budgets[worker.worker_id] - sum(np.round(x[start:start+5])),
        'name': 'worker_unit_budget_%d' % worker.worker_id
    })

#### We now add the deparmental workload constraint, which is dynamic and changes with the assignments in the 'x' vector: 

In [21]:
base_dept_unit_budgets = {
            dept.dept_id: dept.get_remaining_unit_budget(
                project.start_time, project.length
            )
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_ids = base_dept_unit_budgets.keys()
base_dept_unit_budgets = base_dept_unit_budgets

In [22]:
dept_members = {
            dept.dept_id: [m for m in bid_pool if m.department.dept_id==dept.dept_id]
            for dept in set(
                [m.department for m in bid_pool]
            )
        }

dept_members = dept_members

In [23]:
base_dept_unit_budgets

{8: 43.0,
 9: 21.0,
 0: 26.0,
 1: 25.0,
 2: 25.0,
 3: 22.0,
 4: 40.0,
 5: 37.0,
 6: 38.0,
 7: 28.0}

In [24]:
def adjusted_dept_unit_budget(x, dept_id):
    
    base = base_dept_unit_budgets[dept_id]
    for worker in dept_members[dept_id]:
        start = bid_pool.index(worker)
        base -= sum(np.round(x[start:start+5]))
        
    return base

In [25]:
for dept_id in dept_ids:
    constraints.append(
        NonlinearConstraint(lambda x: adjusted_dept_unit_budget(x, dept_id), 0, np.inf)
    )
    constraints_slsqp.append({
        'type': 'ineq', 
        'fun': lambda x: adjusted_dept_unit_budget(x, dept_id),
        'name': 'dept_budget_%d' % dept_id
    })

#### Now we add the team size constraint:

In [26]:
def team_size(x):
    
    size = 0
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        if sum(np.round(x[start:start+5])) > 0:
            size += 1
            
    return size

In [27]:
constraints.append(NonlinearConstraint(team_size, MIN_TEAM_SIZE, MAX_TEAM_SIZE))
#constraints_slsqp = []

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: MAX_TEAM_SIZE - team_size(x), 'name': 'team_size_ub'
})
constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: team_size(x) - MIN_TEAM_SIZE, 'name': 'team_size_lb'
})

#### Add bound constraints (COBYLA only):

In [28]:
#constraints_slsqp = []

for i, xi in enumerate(x):
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: x[i], 'name': 'lb_%d' % i
    })
    constraints_slsqp.append({
        'type': 'ineq', 'fun': lambda x: 1 - x[i] , 'name': 'ub_%d' % i
    })

#### And team budget constraint:

In [29]:
## does bid pool need to be a m.Param?
## does project need to be a m.Param?

def get_team(x):
    
    SKILLS = ['A', 'B', 'C', 'D', 'E']
    contributions = dict()
    #for skill in project.required_skills:
    for skill in SKILLS:
        contributions[skill] = []
    
    team_members = {}
    for wi, worker_id in enumerate(worker_ids):

        start = wi*5
        in_team = False
        
        #for si, skill in enumerate(project.required_skills):
        for si, skill in enumerate(SKILLS):
            if x[start+si] > 0.5:
                contributions[skill].append(worker_id)
                in_team = True
        
        if in_team:
            team_members[worker_id] = bid_pool[wi]
    
    if len(team_members) == 0:
        return Team(project, {}, None)
    else:
        return Team(project, 
                    team_members, 
                    team_members[list(team_members.keys())[0]],
                    contributions=contributions)    

In [30]:
constraints.append(NonlinearConstraint(int(get_team(x).within_budget()), 1, np.inf))
#constraints_slsqp = []

constraints_slsqp.append({
    'type': 'ineq', 'fun': lambda x: -1 + int(get_team(x).within_budget()), 'name': 'budget_constraint'
})


#### Constrain non-required skill elements to zero:

In [31]:
for si, skill in enumerate(['A', 'B', 'C', 'D', 'E']):
    if skill not in project.required_skills:
        
        for i in range(len(worker_ids)):
            constraints_slsqp.append({
                'type': 'ineq', 'fun': lambda x: 0 - x[i*5 + si], 'name': 'non_required_skill_constraint'
            })

In [32]:
constraints = tuple(constraints)

#### Now we define the objective function:

#### Note: we use 0.5 as the boundary to indiciate contributions is non-zero

In [33]:
def objective_func(x):
    
    test_team = get_team(x)
    
    #assert test_team.within_budget()
    
    project.team = test_team
    abm.inventory.success_calculator.calculate_success_probability(project)
    
    return -project.success_probability

# TODO:

- (assign worker contributions in Team! (get_team?))
- test max_team_sze. Increase? Specify equality and check it works - inspect guesses manually

Note: COBYLA ran 'successfully' (before changing 'contributions=contributions' when calling Team()) with maxiter=1000 and ftol=1e-08. But only assigned three elements as non-zero for a project with 6 reuqired skilll units. 

After changing the above, COBYLA also runs 'successfully'...('result')  

'trust-constr' still gives "LinAlgError: SVD did not converge"

'SLSQP' with bespoke bounds (correct?) currently giving exit mode 8 or 5 depending on initial guess.

'COBYLA' finding a solution! But violating constraints so not optimal! Specify bounds as constraints...

#### Note: one option to try is just letting the x vector define team membership (and revert to automatic skill allocation). Then vector is length N.

In [34]:
guess_team = abm.inventory.team_allocator.strategy.select_team(project, bid_pool)

In [35]:
print(guess_team.to_string())

{
    "project": 51,
    "members": [],
    "lead": null,
    "success_probability": 0.0,
    "team_ovr": 0.0,
    "skill_balance": 6.0,
    "creativity_match": 1,
    "skill_contributions": {
        "A": [],
        "B": [],
        "E": []
    }
}


In [36]:
# def get_x(team):
    
#     x = np.zeros(len(bid_pool*5))
#     for member_id in team.members:
#         start = 5*[m.worker_id for m in bid_pool].index(member_id)
#         for si, skill in enumerate(['A', 'B', 'C', 'D', 'E']):
#             if skill in project.required_skills:
#                 x[start+si] = 1
                
#     return x

In [37]:
# method = 'COBYLA' #'COBYLA' #'SLSQP' # 'trust-constr'
# maxiter = 500
# ftol = 1e-09
# eps=1e-8
# catol = 0.0
# tol=1e-2
# rhobeg = 0.6
# verbose=3
# factorization_method='QRFactorization'
# initial_tr_radius=100

# #guess=list([Random.randint(0,1) for i in range(len(x))])
# guess=np.zeros(len(x))
# #guess=np.ones(len(x))
# #guess[1] = 1.0
# #guess[2] = 1.0
# #guess[3] = 1.0
# #guess[4] = 1.0
# #guess[6] = 1.0
# #guess[7] = 1.0
# #guess = get_x(guess_team)

# # For trust-constr: (LinAlgError: SVD did not converge)
# # result_1 = minimize(objective_func, guess, method=method, bounds=bounds, constraints=constraints,
# #                 options={'maxiter': maxiter, 'verbose': verbose, 
# #                          'factorization_method': factorization_method,
# #                          'initial_tr_radius': initial_tr_radius})#,

# # For COBYLA:
# result_1 = minimize(objective_func, guess, method=method,
#                     constraints=constraints_slsqp, options={'maxiter': maxiter, 'disp': True,
#                                                             'catol': catol, 'rhobeg': rhobeg})

In [38]:
# print(team_size(result_1.x))
# print(result_1)
# print(get_team(result_1.x).to_string())

#### Trying basinhopping with COBYLA at each step...

Notes:
- this keeps finding team 1,5,9 with p=0.603
- with niter=1000 it found 5,9,16,31 with p=0.604, took209 seconds
- with niter=10000 it found 1,5,9 with p=0.610, took 2345 seconds (allocation: "B": [5,9],"C": [1,5], "D": [1], "E": [1]
- on second project. with niter=1000, took 629 seconds to find 1,5,6,9. Why same workers?! p=0.647
- on second project. with niter=100, took 67 seconds to find 5,7,63. with p=0.670
- on second project. with niter=100, took 67 seconds to find 1,9,36. with p=0.637
- (attempt 5) third project. with niter=100, took 57 seconds to find 1,3,5,7,17,42,45. with p=0.837
- (attempt 6) third project. with niter=100, took 51 seconds to find 1,3,5,7,17,42,45. with p=0.920 (different contributions)
- (attempt 7) third project. with niter=100, took 47 seconds to find  1,3,5,7,42,59,93. with p=0.72
- (attempt 8) third project. with niter=1000, took 515 seconds to find 1,3,5,7,42,45,65. with p=0.834 
 
- not sure that COBYLA is doing much at each hop?
- try seeding random number generator in SS.utilties for reproducible results
- the returned value does meet constraints, even if COBYLA can't find a local minimum that meets constraints. 

In [39]:
def test_constraints(x, verbose=False):
    
    for cons in constraints_slsqp:
        assert cons['type'] == 'ineq'
        if cons['fun'](x) < 0:
            if verbose:
                print("Constraint violated: %s" % cons['name'])
            return False
        
    return True

In [40]:
class MyConstraints(object):
    
    def __init__(self):
        self.test = test_constraints
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        return self.test(x)

#### Define bespoke step taking:

(Note: without this the basinhopping algorithm perturbs all vector elements by a random amount and result in the team size ub constraint being violated all the time.)

This step function could be made smarter, e.g. by focusing only on required skills. And by choosing the workers to add/remove based on some hueristic rather than randomly...

In [41]:
class MyTakeStep(object):
    
    def __init__(self, bid_pool=bid_pool, project=project): #, stepsize=0.5):
        #self.stepsize = stepsize
        self.name = 'add/remove workers'
        self.bid_pool = bid_pool
        self.skills = ['A', 'B', 'C', 'D', 'E']
        self.project = project
        
        # team needs at least as many members as the maximum required skill units:
        self.min_team_size = max(
            [self.project.get_skill_requirement(skill)['units']
            for skill in self.project.required_skills]
        )
   
    def __call__(self, x):
        
        # determine how many workers to add and remove to team:
        number_to_add = min(Random.randint(0, MAX_TEAM_SIZE), 
                            len(bid_pool) - team_size(x))
        
        new_size = team_size(x) + number_to_add
        min_remove = max(0, new_size - MAX_TEAM_SIZE)
        max_remove = new_size - MIN_TEAM_SIZE
        if max_remove < min_remove:
            number_to_remove = 0
        else:
            number_to_remove = Random.randint(min_remove, max_remove)
        
        # choose members to add:
        assert len(bid_pool) >= number_to_add + number_to_remove
        current_team = get_team(x)
        
        to_add = []
        new_team_members = list(current_team.members.values())
        inverted = bid_pool[::-1]
        for i in range(number_to_add):
            chosen = Random.choice(inverted) 
            while chosen in new_team_members:
                chosen = Random.choice(inverted) 
            new_team_members.append(chosen)
            to_add.append(chosen)
        
        for a in to_add:
            start = bid_pool.index(a) * 5
            
            required_skill_count = len(project.required_skills)
            add_skills = Random.choices(
                project.required_skills, 
                min(required_skill_count, 
                    worker_unit_budgets[a.worker_id])
            )
            for skill in add_skills:
                si = self.skills.index(skill)
                x[start+si] = 1
        
        # now select and remove required numer of workers:
        to_remove = Random.choices(new_team_members, number_to_remove)
        for r in to_remove:
            start = bid_pool.index(r) * 5
            for i in range(5):
                x[start+i] = 0
        
        return x

#### We will run an ensemble of optimisations for the same project/team allocation problem:

- 100 * niter=100 (~30 minutes total)
- 100 * niter=1000 (~3 minutes each -> 5 hours)
- 10 * niter=10000 (~30 minutes each -> 5 hours)
- 1 * niter=100000 (~5hours each).

For each repeat we will save:
- time taken
- optimal objective function
- team (as string)
- success_calculator.to_string(project)

Also save project and bid_pol.

In [42]:
import pickle
RESULTS_DIR = 'experiments/optimisation/'
project_number = 2
bid_pool_number = 2

niter_repeats = {
    100: 100,
    1000: 100,
    10000: 10,
    100000: 1
}

results = {niter: {'time':[], 'prob': []} 
           for niter in niter_repeats.keys()}

In [43]:
with open(RESULTS_DIR + 'bid_pool_%d.json' % bid_pool_number, 'wb') as ofile:
    pickle.dump(bid_pool, ofile)

In [44]:
with open(RESULTS_DIR + 'project_%d.json' % project_number, 'wb') as ofile:
    pickle.dump(project, ofile)

In [45]:
my_constraints = MyConstraints()
my_takestep = MyTakeStep()

maxiter = 100
catol = 0.0
#tol=1e-2
rhobeg = 0.6
niter = 10000

guess=np.zeros(len(x))

minimizer_kwargs = {"method":'COBYLA',
                    'constraints': constraints_slsqp, 
                    'options': {'maxiter': maxiter, 'disp': True, 
                                'catol': catol, 'rhobeg': rhobeg}}

def print_fun(x, f, accepted):
    print("at minimum %.4f accepted %d" % (f, int(accepted)))
    #print(np.where(x==1))
    if accepted:
        print(team_size(x))

In [ ]:
for niter in niter_repeats.keys():
    if niter != 100:
        print('niter = %d' % niter)

        for repeat in range(niter_repeats[niter]):
            print('repeat = %d' % repeat)


            start_time = time.time()
            ret = basinhopping(objective_func, guess, 
                               minimizer_kwargs=minimizer_kwargs,
                               niter=niter, seed=70470, 
                               accept_test=my_constraints,
                               take_step=my_takestep)#,
                               #callback=print_fun,)

            elapsed_time = time.time() - start_time  
            print("%d iterations took %.2f seconds" % (niter, elapsed_time))
            results[niter]['time'].append(elapsed_time)
            results[niter]['prob'].append(ret.fun)

            best_team = get_team(ret.x)
            project.team = best_team
            abm.inventory.success_calculator.calculate_success_probability(project)

            with open(RESULTS_DIR 
                      + 'best_team_project_%d_niter_%d_repeat_%d.json' 
                      % (project_number, niter, repeat), 'wb') as ofile:

                pickle.dump(best_team, ofile)

            with open(RESULTS_DIR 
                      + 'prob_summary_project_%d_niter_%d_repeat_%d.json' 
                      % (project_number, niter, repeat), 'wb') as ofile:

                pickle.dump(
                    abm.inventory.success_calculator.to_string(project), 
                    ofile
                )
        print("Mean prob: ", np.mean(results[niter]['prob']))

niter = 1000
repeat = 0
1000 iterations took 275.46 seconds
repeat = 1
1000 iterations took 280.25 seconds
repeat = 2
1000 iterations took 280.13 seconds
repeat = 3
1000 iterations took 278.05 seconds
repeat = 4
1000 iterations took 265.72 seconds
repeat = 5
1000 iterations took 268.71 seconds
repeat = 6
1000 iterations took 243.35 seconds
repeat = 7
1000 iterations took 230.94 seconds
repeat = 8
1000 iterations took 228.65 seconds
repeat = 9
1000 iterations took 228.65 seconds
repeat = 10
1000 iterations took 227.84 seconds
repeat = 11
1000 iterations took 229.40 seconds
repeat = 12
1000 iterations took 228.99 seconds
repeat = 13
1000 iterations took 228.50 seconds
repeat = 14
1000 iterations took 228.82 seconds
repeat = 15
1000 iterations took 228.43 seconds
repeat = 16
1000 iterations took 228.33 seconds
repeat = 17
1000 iterations took 228.46 seconds
repeat = 18
1000 iterations took 225.23 seconds
repeat = 19
1000 iterations took 227.46 seconds
repeat = 20
1000 iterations took 227.

In [ ]:
print("Mean prob: ", np.mean(results[niter]['prob']))

In [ ]:
with open(RESULTS_DIR 
                  + 'all_results_project_%d.json' 
                  % project_number, 'wb') as ofile:
                
            pickle.dump(
                results, 
                ofile
            )

In [182]:
best_team = get_team(ret.x)
project.team = best_team
abm.inventory.success_calculator.calculate_success_probability(project)

In [183]:
print(abm.inventory.success_calculator.to_string(project))

{
    "ovr (value, prob): ": [
        46.86307715955965,
        35.14730786966974
    ],
    "skill balance (value, prob): ": [
        1.339505081334679,
        -3.348762703336697
    ],
    "creativity match (value, prob): ": [
        0.020249732948424765,
        9.198156638611469
    ],
    "risk (value, prob): ": [
        25,
        -15.0
    ],
    "chemistry (value, prob): ": [
        2.0,
        5.0
    ]
}


In [185]:
print(best_team.to_string())

{
    "project": 51,
    "members": [
        13,
        39,
        96,
        103,
        104
    ],
    "lead": 13,
    "success_probability": 0.3,
    "team_ovr": 46.9,
    "skill_balance": 1.3,
    "creativity_match": 0.0,
    "skill_contributions": {
        "A": [
            104
        ],
        "B": [
            39,
            104
        ],
        "C": [
            13,
            39,
            96,
            103,
            104
        ],
        "D": [
            13,
            39,
            103,
            104
        ],
        "E": []
    }
}


In [131]:
print(bid_pool[[m.worker_id for m in bid_pool].index(118)].skills.to_string())

{
    "Worker OVR": 68.1,
    "Hard skills": [
        4.2,
        0.0,
        2.1,
        3.8,
        3.6
    ],
    "Soft skills": [
        1.7,
        2.5,
        4.2,
        2.6,
        4.8
    ],
    "Hard skill probability": 0.8,
    "OVR multiplier": 20
}


In [154]:
print(project.requirements.to_string())

{
    "risk": 25,
    "creativity": 4,
    "flexible_budget": false,
    "budget": 47,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 16,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 1,
            "units": 3
        },
        "C": {
            "level": 5,
            "units": 6
        },
        "D": {
            "level": 2,
            "units": 7
        },
        "E": {
            "level": null,
            "units": 0
        }
    }
}


In [148]:
print(project.requirements.to_string())

{
    "risk": 25,
    "creativity": 5,
    "flexible_budget": true,
    "budget": 38.75,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 19,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 3,
            "units": 4
        },
        "C": {
            "level": 2,
            "units": 4
        },
        "D": {
            "level": 1,
            "units": 6
        },
        "E": {
            "level": 1,
            "units": 5
        }
    }
}


In [73]:
attempt = 9

*attempt_1 (project 1, niter=10000)
* attempt_2  (project 2, niter=1000)
* [attempt_3  (project 2, niter=100)]
* attempt_4  (project 2, niter=100)
* attempt_5  (project 3, niter=100) *New system state. ABM recreated and run for 25 steps. Bid pool of 39.
* attempt_6  (project 3, niter=100). Same team as attempt 5!
* attempt_7 (projet 3, niter=100). Worse.


#### Note that our methods seems to favour lower numbers (1,5,7,9)? Tried new abm to determine if this was to do with specific workers...

In [704]:
best_team.within_budget()

False

In [93]:
print(project.requirements.to_string())

{
    "risk": 5,
    "creativity": 1,
    "flexible_budget": false,
    "budget": 25,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 6,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 5,
            "units": 2
        },
        "C": {
            "level": 5,
            "units": 2
        },
        "D": {
            "level": 2,
            "units": 1
        },
        "E": {
            "level": 3,
            "units": 1
        }
    }
}


In [97]:
sum(result.x)

4.016970683373013

In [98]:
objective_func(result.x)

-0.5078404640292967

In [99]:
best_team = get_team(result.x)
project.team = best_team
abm.inventory.success_calculator.calculate_success_probability(project)

In [100]:
print(best_team.to_string())

{
    "project": 31,
    "members": [
        5,
        9,
        45
    ],
    "lead": 5,
    "success_probability": 0.5,
    "team_ovr": 84.1,
    "skill_balance": 5.5,
    "creativity_match": 0.4,
    "skill_contributions": {
        "A": [
            45
        ],
        "B": [
            5,
            9
        ],
        "C": [
            5
        ],
        "D": [],
        "E": []
    }
}


In [110]:
with open('project_attempt_6.json', 'rb') as ifile:
    temp = pickle.load(ifile)
    
print(temp.requirements.to_string())

{
    "risk": 25,
    "creativity": 5,
    "flexible_budget": true,
    "budget": 38.75,
    "p_hard_skill_required": 0.8,
    "min_skill_required": 2,
    "per_skill_cap": 7,
    "total_skill_units": 19,
    "hard_skills": {
        "A": {
            "level": null,
            "units": 0
        },
        "B": {
            "level": 3,
            "units": 4
        },
        "C": {
            "level": 2,
            "units": 4
        },
        "D": {
            "level": 1,
            "units": 6
        },
        "E": {
            "level": 1,
            "units": 5
        }
    }
}


In [112]:
with open('best_team_attempt_6.json', 'rb') as ifile:
    temp = pickle.load(ifile)
    
print(temp)

{
    "project": 51,
    "members": [
        1,
        3,
        5,
        7,
        17,
        42,
        45
    ],
    "lead": 1,
    "success_probability": 0.9,
    "team_ovr": 127.6,
    "skill_balance": 0.9,
    "creativity_match": 0.1,
    "skill_contributions": {
        "A": [
            1,
            3,
            5,
            7,
            17
        ],
        "B": [
            5
        ],
        "C": [
            45
        ],
        "D": [
            3,
            42
        ],
        "E": [
            5
        ]
    }
}


In [84]:
type(best_team.project.success_probability)

numpy.float64

In [86]:
best_team.round_to

{'A': [1, 5, 6], 'B': [], 'C': [], 'D': [], 'E': []}